In [313]:
import pathlib
import pandas as pd
import re
import bs4
import ctypes
import requests
import time
from bs4 import BeautifulSoup
from unidecode import unidecode

In [314]:
path_processed = pathlib.Path("/export/usuarios_ml4ds/lbartolome/NextProcurement/NP-Search-Tool/sample_data/all_processed/md_sin_lote_es.parquet")
path_raw = pathlib.Path("/export/usuarios_ml4ds/lbartolome/NextProcurement/NP-Search-Tool/sample_data/all_processed/minors_insiders_outsiders_origen_sin_lot_info.parquet")

In [315]:
df_processed = pd.read_parquet(path_processed)
df_raw = pd.read_parquet(path_raw)
df = pd.merge(df_processed, df_raw, how='inner', on='id_tm')
df

,id_tm,raw_text,lemmas,lang,origen,title,summary,text
0,0,Reforma de elementos de ventilación exterior de equipo de aire acondicionado del Teatro Villa de Molina y modificación de caudal de aire de equipos para mejorar la temperatura y confort del centro. Id licitación: 000103/2017-1069; Órgano de Contratación: Junta de Gobierno del Ayuntamiento de Molina de Segura; Importe: 5100 EUR; Estado: RES,reforma elemento ventilación exterior aire_acondicionado teatro villa molina modificación caudal aire mejorar temperatura confort id contratación junta gobierno ayuntamiento molina eur res,es,minors,Reforma de elementos de ventilación exterior de equipo de aire acondicionado del Teatro Villa de Molina y modificación de caudal de aire de equipos para mejorar la temperatura y confort del centro,Id licitación: 000103/2017-1069; Órgano de Contratación: Junta de Gobierno del Ayuntamiento de Molina de Segura; Importe: 5100 EUR; Estado: RES,Reforma de elementos de ventilación exterior de equipo de aire acondicionado del Teatro Villa de Molina y modificación de caudal de aire de equipos para mejorar la temperatura y confort del centro. Id licitación: 000103/2017-1069; Órgano de Contratación: Junta de Gobierno del Ayuntamiento de Molina de Segura; Importe: 5100 EUR; Estado: RES
1,1,Servicios de calibrado y certificado de dos detectores portátiles de gases. Trasvase reversible Ebro-Besaya.. Id licitación: 29-2017-II; Órgano de Contratación: Presidencia de la Confederación Hidrográfica del Cantábrico; Importe: 240 EUR; Estado: RES,servicios calibrado certificado detector portátil gas trasvase_reversible id contratación presidencia confederación_hidrográfica cannotábrico eur res,es,minors,Servicios de calibrado y certificado de dos detectores portátiles de gases. Trasvase reversible Ebro-Besaya.,Id licitación: 29-2017-II; Órgano de Contratación: Presidencia de la Confederación Hidrográfica del Cantábrico; Importe: 240 EUR; Estado: RES,Servicios de calibrado y certificado de dos detectores portátiles de gases. Trasvase reversible Ebro-Besaya.. Id licitación: 29-2017-II; Órgano de Contratación: Presidencia de la Confederación Hidrográfica del Cantábrico; Importe: 240 EUR; Estado: RES
2,2,"Patrocinio menor proyecto "" Activitats C.I.N.E"". Id licitación: 013-07-2018; Órgano de Contratación: Agencia de Turismo de las Illes Balears; Importe: 8866.11 EUR; Estado: RES",patrocinio activitats id contratación agencia turismo illes_balears eur res,es,minors,"Patrocinio menor proyecto "" Activitats C.I.N.E""",Id licitación: 013-07-2018; Órgano de Contratación: Agencia de Turismo de las Illes Balears; Importe: 8866.11 EUR; Estado: RES,"Patrocinio menor proyecto "" Activitats C.I.N.E"". Id licitación: 013-07-2018; Órgano de Contratación: Agencia de Turismo de las Illes Balears; Importe: 8866.11 EUR; Estado: RES"
3,3,Redacción EPIA - Legalización antena emisora de Radio Narcea. Id licitación: CON/2017/51; Órgano de Contratación: Alcaldía del Ayuntamiento de Cangas del Narcea; Importe: 2000 EUR; Estado: RES,epio legalización antena_emisor radio narcea id contratación alcaldía ayuntamiento narcea eur res,es,minors,Redacción EPIA - Legalización antena emisora de Radio Narcea,Id licitación: CON/2017/51; Órgano de Contratación: Alcaldía del Ayuntamiento de Cangas del Narcea; Importe: 2000 EUR; Estado: RES,Redacción EPIA - Legalización antena emisora de Radio Narcea. Id licitación: CON/2017/51; Órgano de Contratación: Alcaldía del Ayuntamiento de Cangas del Narcea; Importe: 2000 EUR; Estado: RES
4,4,Obras de reparación del Centro de Información de la Naturaleza de El Rellano. Id licitación: 000047/2017-1069; Órgano de Contratación: Junta de Gobierno del Ayuntamiento de Molina de Segura; Importe: 3957.95 EUR; Estado: RES,centro información naturaleza rellano id contratación junta gobierno ayuntamiento molina eur res,es,minors,Obras de reparación del Centro de Información de la Naturaleza de El Rellano,Id licitación: 000047/2017-1069; Órgano de Contratación: Junta de G

In [316]:
df.iloc[0].title

'Reforma de elementos de ventilación exterior de equipo de aire acondicionado del Teatro Villa de Molina y modificación de caudal de aire de equipos para mejorar la temperatura y confort del centro'

In [317]:
df.iloc[0].summary

'Id licitación: 000103/2017-1069; Órgano de Contratación: Junta de Gobierno del Ayuntamiento de Molina de Segura; Importe: 5100 EUR; Estado: RES'

In [204]:
df[df.id_tm == 2989227].summary.values.tolist()[0]

'Id licitación:  +6.6II45TA \n ; Órgano de contratación: Servicio Andaluz de Salud;'

In [243]:
df_aux = df

#### Remove Id licitación:.*?Órgano de Contratación:

In [247]:
pattern = (
    r'|;\s*Número de\s+Estado: RES\s*'
    r'|;\s*Número de\s+Estado: EV\s*'
    r'|;\s*Número de\s*'
    r'|;\s*Id licitación:.*?Estado: RES\s*'
    r'|;\s*Id licitación:.*?Estado: En Plazo\s*'
    r'|;\s*Id licitación:.*?Estado: ADJ\s*'
    r'|;\s*Id licitación:.*?Estado: PUB\s*'
    r'|;\s*Id licitación:.*?Estado: EV\s*'
    r'|;\s*Id licitación:.*?Estado: ANUL\s*'
    r'|;\s*Id licitación:.*?Estado: PRE\s*'
    r'|;\s*Id licitación:.*?Estado: CERR\s*'
    r'|;\s*Id licitación:.*?Órgano de Contratación:.*?;\s*'
    r'|Id licitación:.*?Órgano de Contratación:.*?;'
    r'|;\s*Id licitación:.*?Importe.*?€;\s*'
    r'|;\s*Id licitación:.*?Importe.*?,\s*'
    r'|;\s*Órgano de Contratación:.*?Estado: PRE\s*'
    r'|;\s*Importe:.*?Estado: CERR\s*'
    r'|;\s*Importe:.*?Estado: BORR\s*'
    r'|;\s*Importe:.*?EUR;\s*'
    r'|;\s*Expediente:.*?Resuelta\s*'
    r'|;\s*Expediente:.*?En Plazo\s*'
    r'|;\s*Expediente:.*?En prazo\s*'
    r'|;\s*Expediente:.*?Adj\s*'
    r'|;\s*Expediente:.*?Órgano de Contratación:.*?;\s*'
    r'|Estado: PENDIENTE'
    r'|Estado: ABIERTA'
    r'|Estado:EN PLAZO'
    r'|;\s*Estado.*?ADJUDICACION\s*'
    r'|;\s*Estado.*?ADJUDICACIÓN\s*'
    r'|;\s*Estado: 1Abierta\s*'
    r'|;\s*Estado: 2Entransicion\s*'
    r'|;\s*Estado: 3Cerrada\s*'
    r'|;\s*Estado: En prazo\s*'
    r'|;\s*Estado: Anuncio Previo\s*'
    r'|;\s*Estado: Anul\s*'
    r'|;\s*Estado:\s+RES\s*'
    r'|;\s*Estado: Formalización de contratos\s*'
    r'|Estado: Apertura documentación técnica'
    r'|Estado: No formalizado'
    r'|Estado: Clasificación de ofertas'
    r'|Estado: Empresas seleccions'
    r'|;\s*Estado: ABIERTO\s*'
    r'|Estado:ADJ'
    r'|Estado:ANUL'
    r'|;\s*ALUACION\s*'
    r'|UELTA'
    r'|udicada'
    r'|aluación'
    r'|udicación'
    r'|de presentación'
    r'|ada'
    r'|uelto'
    r'|Pleno'
    r'|;\s*Importe:.*?;\s*'
    r'|licación en Perfil del Contratante'
    r'|Id licitación:\s*\+?\d+(\.\d+)?[A-Z\d\-]+[\s\n]*; Órgano de contratación: Servicio Andaluz de Salud;'
    r'|Órgano de Contratación:\s*[\w\s\',.\-\(\)àèéíòóúçÁÉÍÓÚÀÈÇ]+\s*;'
    r'|Id licitación:\s*\+?[A-Z\d\-]+[\s\n]*;.*?'
    r'|Expediente:\s*\d+\/\d+\(.*?\),\s*Entidad:.*?,\s*Importe:\s*\d+\.\d+\s*,'
    r'|Estado: Apertura documentación administrativa'
)

df_aux['summary'] = df_aux['summary'].str.replace(pattern, '', regex=True, flags=re.IGNORECASE)


In [248]:
non_empty_rows = df_aux['summary'].str.strip() != ''
non_empty_df = df_aux[non_empty_rows]
print(f"There are {len(non_empty_df)} / {len(df_aux)} after removing 'Id licitación:.*?Órgano de Contratación:'")
non_empty_df

There are 8 / 2944105 after removing 'Id licitación:.*?Órgano de Contratación:'


,id_tm,raw_text,lemmas,lang,origen,title,summary,text,title_split
2787619,2818055,Enajenación de la parcela A de las resultantes...,enajenación_parcela resultant proyecto_reparce...,es,outsiders,Enajenación de la parcela A de las resultantes...,Gobierno de Zaragoza,Enajenación de la parcela A de las resultantes...,"[Enajenación, de, la, parcela, A, de, las, res..."
2788749,2820316,Obras de impermeabilizaciÃ³n y reforma Piscina...,obras reforma piscinas expediente entidad ayun...,es,outsiders,Obras de impermeabilizaciÃ³n y reforma Piscinas,"Expediente: 022/2018, Entidad: Ayuntamiento de...",Obras de impermeabilizaciÃ³n y reforma Piscina...,"[Obras, de, impermeabilizaciÃ³n, y, reforma, P..."
2789171,2821229,ContrataciÃ³n del servicio de bar en el cdm sa...,cdm isabel id concejal_delegado vivienda depor...,es,outsiders,ContrataciÃ³n del servicio de bar en el cdm sa...,Concejal Delegado de Vivienda y Deporte del A...,ContrataciÃ³n del servicio de bar en el cdm sa...,"[ContrataciÃ³n, del, servicio, de, bar, en, el..."
2790022,2822745,Aprovehamiento de resinas en Monte CUP nº 39 ....,aprovehamiento resina monte_cup nº expediente ...,es,outsiders,Aprovehamiento de resinas en Monte CUP nº 39 ....,"Expediente: 81/2018, Entidad: Ayuntamiento de ...",Aprovehamiento de resinas en Monte CUP nº 39 ....,"[Aprovehamiento, de, resinas, en, Monte, CUP, ..."
2790120,2822930,Libreria de cintas lto. Id licitación: 0055650...,libreria_cinta lto id consejera participación_...,es,outsiders,Libreria de cintas lto,Consejera de Participación Transparencia y Go...,Libreria de cintas lto. Id licitación: 0055650...,"[Libreria, de, cintas, lto]"
2791919,2826639,Gestión del servicio de explotación integral d...,gestión explotación integral piscina temporada...,es,outsiders,Gestión del servicio de explotación integral d...,"Expediente: 109/2018, Entidad: Ayuntamiento de...",Gestión del servicio de explotación integral d...,"[Gestión, del, servicio, de, explotación, inte..."
2792798,2828507,"Alquiler, mantenimiento integral y seguro de a...",alquiler integral seguro bicicleta bicicleta b...,es,outsiders,"Alquiler, mantenimiento integral y seguro de a...",Consejero de Urbanismo y Sostenibilidad,"Alquiler, mantenimiento integral y seguro de a...","[Alquiler,, mantenimiento, integral, y, seguro..."
2878956,2996515,contrato basado en acuerdo marco con\nuna únic...,único_fijar condición_tracto sucesivo_precio u...,es,outsiders,contrato basado en acuerdo marco con una única...,Id licitación: +6.C+D45BA\n ;,contrato basado en acuerdo marco con\nuna únic...,"[contrato, basado, en, acuerdo, marco, con, un..."


In [250]:
non_empty_df.summary.values.tolist()

[' Gobierno de Zaragoza',
 'Expediente: 022/2018, Entidad: Ayuntamiento de Alcazaren, Importe: 60207.61,',
 ' Concejal Delegado de Vivienda y Deporte del Ayuntamiento de Zaragoza',
 'Expediente: 81/2018, Entidad: Ayuntamiento de Olmedo, Importe: 32988.00,',
 ' Consejera de Participación Transparencia y Gobierno abierto del Ayuntamiento de Zaragoza',
 'Expediente: 109/2018, Entidad: Ayuntamiento de Simancas, Importe: 6000.00,',
 ' Consejero de Urbanismo y Sostenibilidad',
 'Id licitación: +6.C+D45BA\n ; ']

#### Keep only words in dictionary

In [303]:
host = 'http://dle.rae.es'
referer = 'http://www.rae.es/'
parser = 'lxml'
s = requests.Session()

def search(word):
    s.headers.update({'Upgrade-Insecure-Requests': '1'})
    s.headers.update({'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8'})
    s.headers.update({'Referer': referer})
    s.headers.update({'Accept-Language': 'es-ES,es;q=0.8'})
    s.headers.update({'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.81 Safari/537.36'})
    s.headers.update({'Accept-Encoding': 'gzip, deflate, sdch'})

    s.cookies = requests.utils.cookiejar_from_dict({'cookies_rae': 'aceptada'})

    url1 = host + '/?w=' + word
    #print(url1)
    url2 = host + '/srv/search?w=' + word

    response = s.get(url1).text
    soup = BeautifulSoup(response, parser)
    text = soup.get_text()
    is_in_dict = f"La palabra {word} no está en el Diccionario"
    if is_in_dict in text:
        return False
    else:
        return True

In [301]:
search("González")

http://dle.rae.es/?w=González


False

In [269]:
path_apellidos = "apellidos.parquet"
path_nombres = "nombres.parquet"

# Load data using pandas
df_apellidos = pd.read_parquet(path_apellidos)
df_nombres = pd.read_parquet(path_nombres)

# Apply transformations to split composed names
df_apellidos["palabras"] = df_apellidos["apellidos"].str.split()
df_apellidos = df_apellidos.explode("palabras").rename(columns={"palabras": "apellidos_split"})

df_nombres["palabras"] = df_nombres["nombres"].str.split()
df_nombres = df_nombres.explode("palabras").rename(columns={"palabras": "nombres_split"})
print(df_nombres)

# Get unique names and last names
unique_apellidos = df_apellidos["apellidos_split"].unique()
unique_nombres = df_nombres["nombres_split"].unique()
unique_names_apellidos = list(set(unique_nombres) | set(unique_apellidos))


               nombres nombres_split
0              antonio       antonio
1               manuel        manuel
2                 jose          jose
3            francisco     francisco
4                david         david
...                ...           ...
55759          zobeida       zobeida
55760   zulay josefina         zulay
55760   zulay josefina      josefina
55761  zulma elizabeth         zulma
55761  zulma elizabeth     elizabeth

[92066 rows x 2 columns]


In [291]:
"hospital" in unique_names_apellidos

True

In [284]:
unidecode("García".lower())

'garcia'

In [305]:
df_sample = df_aux.iloc[0:10]

In [306]:
df_sample["title_split"] = df_sample["title"].apply(lambda x: " ".join([x for x in x.split() if search(x)]))
df_sample[["title", "title_split"]]

/tmp/ipykernel_1069691/4215601762.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sample["title_split"] = df_sample["title"].apply(lambda x: " ".join([x for x in x.split() if search(x)]))


,title,title_split
0,Reforma de elementos de ventilación exterior de equipo de aire acondicionado del Teatro Villa de Molina y modificación de caudal de aire de equipos para mejorar la temperatura y confort del centro,Reforma de elementos de ventilación exterior de equipo de aire acondicionado del Teatro Villa de y modificación de caudal de aire de equipos para mejorar la temperatura y confort del centro
1,Servicios de calibrado y certificado de dos detectores portátiles de gases. Trasvase reversible Ebro-Besaya.,Servicios de calibrado y certificado de dos detectores portátiles de Trasvase reversible
2,"Patrocinio menor proyecto "" Activitats C.I.N.E""","Patrocinio menor proyecto "" C.I.N.E"""
3,Redacción EPIA - Legalización antena emisora de Radio Narcea,Redacción Legalización antena emisora de Radio
4,Obras de reparación del Centro de Información de la Naturaleza de El Rellano,Obras de reparación del Centro de Información de la Naturaleza de El Rellano
5,"Diversos materiales para el acondicionamiento de rotondas, arriates y decoracion de jardineras y macetas en la via publica",Diversos materiales para el acondicionamiento de arriates y de jardineras y macetas en la publica
6,Adquisición de un espectrómetro de masas con destino al Laboratorio de Calidad de Aguas de la Confederación Hidrográfica del Ebro,Adquisición de un espectrómetro de masas con destino al Laboratorio de Calidad de Aguas de la Confederación Hidrográfica del
7,Suministro Lácteos Y Derivados Para Hospital Mateu Orfila,Suministro Lácteos Y Derivados Para Hospital
8,"Actuaciones de mejora de las condiciones de seguridad, confort y eficiencia energética en edificio destinado a Biblioteca Municipal ""Salvador García Aguilar""","Actuaciones de mejora de las condiciones de confort y eficiencia energética en edificio destinado a Biblioteca Municipal ""Salvador García Aguilar"""
9,Suministro De Armarios Para La Unidad De Bolsa En Joan Munar (SSCC),Suministro De Armarios Para La Unidad De Bolsa En (SSCC)


In [307]:
pd.set_option('display.max_colwidth', None)
df_sample[["title", "title_split"]]

,title,title_split
0,Reforma de elementos de ventilación exterior de equipo de aire acondicionado del Teatro Villa de Molina y modificación de caudal de aire de equipos para mejorar la temperatura y confort del centro,Reforma de elementos de ventilación exterior de equipo de aire acondicionado del Teatro Villa de y modificación de caudal de aire de equipos para mejorar la temperatura y confort del centro
1,Servicios de calibrado y certificado de dos detectores portátiles de gases. Trasvase reversible Ebro-Besaya.,Servicios de calibrado y certificado de dos detectores portátiles de Trasvase reversible
2,"Patrocinio menor proyecto "" Activitats C.I.N.E""","Patrocinio menor proyecto "" C.I.N.E"""
3,Redacción EPIA - Legalización antena emisora de Radio Narcea,Redacción Legalización antena emisora de Radio
4,Obras de reparación del Centro de Información de la Naturaleza de El Rellano,Obras de reparación del Centro de Información de la Naturaleza de El Rellano
5,"Diversos materiales para el acondicionamiento de rotondas, arriates y decoracion de jardineras y macetas en la via publica",Diversos materiales para el acondicionamiento de arriates y de jardineras y macetas en la publica
6,Adquisición de un espectrómetro de masas con destino al Laboratorio de Calidad de Aguas de la Confederación Hidrográfica del Ebro,Adquisición de un espectrómetro de masas con destino al Laboratorio de Calidad de Aguas de la Confederación Hidrográfica del
7,Suministro Lácteos Y Derivados Para Hospital Mateu Orfila,Suministro Lácteos Y Derivados Para Hospital
8,"Actuaciones de mejora de las condiciones de seguridad, confort y eficiencia energética en edificio destinado a Biblioteca Municipal ""Salvador García Aguilar""","Actuaciones de mejora de las condiciones de confort y eficiencia energética en edificio destinado a Biblioteca Municipal ""Salvador García Aguilar"""
9,Suministro De Armarios Para La Unidad De Bolsa En Joan Munar (SSCC),Suministro De Armarios Para La Unidad De Bolsa En (SSCC)


In [308]:
df_sample.columns

Index(['id_tm', 'raw_text', 'lemmas', 'lang', 'origen', 'title', 'summary',
       'text', 'title_split'],
      dtype='object')

In [309]:
df_sample[['id_tm', 'origen', 'title', 'title_split']]

,id_tm,origen,title,title_split
0,0,minors,Reforma de elementos de ventilación exterior de equipo de aire acondicionado del Teatro Villa de Molina y modificación de caudal de aire de equipos para mejorar la temperatura y confort del centro,Reforma de elementos de ventilación exterior de equipo de aire acondicionado del Teatro Villa de y modificación de caudal de aire de equipos para mejorar la temperatura y confort del centro
1,1,minors,Servicios de calibrado y certificado de dos detectores portátiles de gases. Trasvase reversible Ebro-Besaya.,Servicios de calibrado y certificado de dos detectores portátiles de Trasvase reversible
2,2,minors,"Patrocinio menor proyecto "" Activitats C.I.N.E""","Patrocinio menor proyecto "" C.I.N.E"""
3,3,minors,Redacción EPIA - Legalización antena emisora de Radio Narcea,Redacción Legalización antena emisora de Radio
4,4,minors,Obras de reparación del Centro de Información de la Naturaleza de El Rellano,Obras de reparación del Centro de Información de la Naturaleza de El Rellano
5,5,minors,"Diversos materiales para el acondicionamiento de rotondas, arriates y decoracion de jardineras y macetas en la via publica",Diversos materiales para el acondicionamiento de arriates y de jardineras y macetas en la publica
6,6,minors,Adquisición de un espectrómetro de masas con destino al Laboratorio de Calidad de Aguas de la Confederación Hidrográfica del Ebro,Adquisición de un espectrómetro de masas con destino al Laboratorio de Calidad de Aguas de la Confederación Hidrográfica del
7,7,minors,Suministro Lácteos Y Derivados Para Hospital Mateu Orfila,Suministro Lácteos Y Derivados Para Hospital
8,8,minors,"Actuaciones de mejora de las condiciones de seguridad, confort y eficiencia energética en edificio destinado a Biblioteca Municipal ""Salvador García Aguilar""","Actuaciones de mejora de las condiciones de confort y eficiencia energética en edificio destinado a Biblioteca Municipal ""Salvador García Aguilar"""
9,9,minors,Suministro De Armarios Para La Unidad De Bolsa En Joan Munar (SSCC),Suministro De Armarios Para La Unidad De Bolsa En (SSCC)
